In [1]:
%%capture
# !pip install llama-index==0.10.37 html2text

In [2]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

# 📂 **Loading Data**

Preparing your data for an LLM involves an ingestion pipeline similar to ML data cleaning or traditional ETL processes.

### **Ingestion Pipeline Stages**
  - 📥 Load the data
  - 🔧 Transform the data
  - 🗃️ Index and store the data


Let's start by downloading some example files

In [3]:
import requests
from pathlib import Path

# Base URL for Project Gutenberg texts
base_url = "https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt"

# Directory to save the downloaded files
directory = Path("./data")

# Create the directory if it doesn't exist
directory.mkdir(parents=True, exist_ok=True)

# Generate a list of book IDs to download
book_ids = range(1, 11)  # This will create a range from 1 to 10

# Generate URLs for each book ID
urls = [base_url.format(book_id=book_id) for book_id in book_ids]

# Download each file and save it in the specified directory
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        # Extract the filename from the URL using the book ID and create a file name
        book_id = url.split('/')[-2]  # Extracts the book ID from the URL
        filename = f"pg{book_id}.txt"
        file_path = directory / filename
        # Save the file to the specified directory
        file_path.write_text(response.text)
        print(f"Downloaded {filename} to {file_path}")
    else:
        print(f"Failed to download {url}. HTTP status code: {response.status_code}")

Downloaded pg1.txt to data/pg1.txt
Downloaded pg2.txt to data/pg2.txt
Downloaded pg3.txt to data/pg3.txt
Downloaded pg4.txt to data/pg4.txt
Downloaded pg5.txt to data/pg5.txt
Downloaded pg6.txt to data/pg6.txt
Downloaded pg7.txt to data/pg7.txt
Downloaded pg8.txt to data/pg8.txt
Downloaded pg9.txt to data/pg9.txt
Downloaded pg10.txt to data/pg10.txt


# 📥 Load the data

To use data with an LLM, first load it using data connectors, known as `Readers` in LlamaIndex, which format data into `Document` objects containing data and metadata.

📚 **SimpleDirectoryReader**:
  - The most straightforward loader is `SimpleDirectoryReader``.
  - Built into LlamaIndex, it reads various formats (Markdown, PDFs, Word documents, PowerPoint decks, images, audio, video) from every file in a directory, creating documents.

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/lil_llama_index/lib/python3.10/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
len(documents)

11

In [6]:
type(documents[0])

llama_index.core.schema.Document

In [7]:
documents[3].__dict__

{'id_': '28136181-1669-4723-a9e3-0970f46d2897',
 'embedding': None,
 'metadata': {'file_path': '/workspaces/hands-on-ai-rag-using-llamaindex-3830207/02_Fundamental_Concepts_in_LlamaIndex/data/pg2.txt',
  'file_name': 'pg2.txt',
  'file_type': 'text/plain',
  'file_size': 30938,
  'creation_date': '2025-07-24',
  'last_modified_date': '2025-07-24'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'text': '\ufeffThe Project Gutenberg eBook of The United States Bill of Rights\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg 

##### Manually Create Document Objects

In [8]:
from llama_index.core import Document

manual_document = Document(text="This is an example of a manual document")

In [9]:
manual_document.__dict__

{'id_': '945724c0-a532-47ff-95f5-84dc391a3efb',
 'embedding': None,
 'metadata': {},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'text': 'This is an example of a manual document',
 'mimetype': 'text/plain',
 'start_char_idx': None,
 'end_char_idx': None,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template': '{key}: {value}',
 'metadata_seperator': '\n'}

##### Adding metadata

You can add metadata in the document constructor:

In [10]:
manual_document_with_metadata = Document(
    text="This is an example of a manual document",
    metadata={"filename": "made-up-file-name", "category": "imaginary-category"}
)

In [11]:
manual_document_with_metadata.__dict__

{'id_': '8a2f2faf-fdb8-4cc9-b267-d61752c365e6',
 'embedding': None,
 'metadata': {'filename': 'made-up-file-name',
  'category': 'imaginary-category'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'text': 'This is an example of a manual document',
 'mimetype': 'text/plain',
 'start_char_idx': None,
 'end_char_idx': None,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template': '{key}: {value}',
 'metadata_seperator': '\n'}

Or after the document is created

In [12]:
manual_document.metadata={"filename": "made-up-file-name", "category": "imaginary-category"}

In [13]:
manual_document.__dict__

{'id_': '945724c0-a532-47ff-95f5-84dc391a3efb',
 'embedding': None,
 'metadata': {'filename': 'made-up-file-name',
  'category': 'imaginary-category'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'text': 'This is an example of a manual document',
 'mimetype': 'text/plain',
 'start_char_idx': None,
 'end_char_idx': None,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template': '{key}: {value}',
 'metadata_seperator': '\n'}

# 🔧 Transform the data

After loading, we must process and transform data for retrieval. We need to transform the list of `Document` objects into `Node` objects 

- ✂️ Include chunking, extracting metadata, and embedding each chunk in transformations.

- 🌟 Nodes are a first-class citizen in LlamaIndex, allowing direct definition or parsing from Documents.

- 🔄 Transformation inputs and outputs are `Node` objects (Note: `Document` is subclass of `Node`).

- 🛠️ Nodes are "chunks" of Documents, including text, images, etc., plus metadata and relationships.

- 📊 `NodeParser` classes convert Documents into Nodes with all necessary attributes. There are [a number of](https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules.html) `NodeParser`'s you can choose from!

- 📑 High-level API: Use `.from_documents()` for automatic parsing and chunking of Document objects.

- 🔍 Underlying process splits Document into Node objects, maintaining text and metadata with a link to their parent Document.


In [14]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(
    chunk_size=128, # in tokens
    chunk_overlap=16, #in tokens
    paragraph_separator="\n\n"
)

nodes = parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/11 [00:00<?, ?it/s]

In [15]:
type(nodes[42])

llama_index.core.schema.TextNode

In [16]:
nodes[42].__dict__

{'id_': '667c8ee8-6e19-4b7a-a164-613ee513dd77',
 'embedding': None,
 'metadata': {'file_path': '/workspaces/hands-on-ai-rag-using-llamaindex-3830207/02_Fundamental_Concepts_in_LlamaIndex/data/pg1.txt',
  'file_name': 'pg1.txt',
  'file_type': 'text/plain',
  'file_size': 127192,
  'creation_date': '2025-07-24',
  'last_modified_date': '2025-07-24'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4b664c0b-b83e-448c-ae2d-e28cbfdc86c6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/workspaces/hands-on-ai-rag-using-llamaindex-3830207/02_Fundamental_Concepts_in_LlamaIndex/data/pg1.txt', 'file_name': 'pg1.txt', 'file_type': 'text/plain', 'file_size': 127192, 'crea

You can also choose to construct Node objects manually.


In [17]:
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo

node1 = TextNode(text="Dad is married to Mom", id_="001")

node2 = TextNode(text="Dad is Son's dad", id_="002")

## NodeRelationships

You can set relationships between nodes.

- 🌐 NodeRelationships assign connections between chunks of text. It's useful for:
  - Documents organized in a hierarchical manner (e.g., book, chapter, section, subsection)
  - Maintaining sequential order
  - Other complex relationships (ie, in legal documents for links a clause or other cases) 

- 🔍 NodeRelationships help retrieve not just the relevant section, but also related sections that might provide additional context or information.

In [18]:
node1.relationships[NodeRelationship.NEXT] = RelatedNodeInfo(
    node_id=node2.node_id
)

node2.relationships[NodeRelationship.PREVIOUS] = RelatedNodeInfo(
    node_id=node1.node_id
)
nodes = [node1, node2]

node2.relationships[NodeRelationship.PARENT] = RelatedNodeInfo(
    node_id=node1.node_id, metadata={"Romie": "Mom", "Harpreet": "Dad", "Jind":"Daughter", "Jugaad":"Son"}, 
)

A bit of clean up, let's just go ahead and delete the text files we downloaded since we won't need them going forward.


In [19]:
# !rm -rf ./data